In [3]:
!pip install pyspark
!pip install findspark

In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import findspark
findspark.init

from pyspark.sql import SparkSession # required to created a dataframe
spark=SparkSession.builder.appName("Basics").getOrCreate() 

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
file = '../input/bitcoin-tweets-20160101-to-20190329/tweets.csv'
df = spark.read.csv(file, sep=";",inferSchema=True, header=True,multiLine=True)

In [22]:
df.show(10)

+-------------------+---------------+-------------------+----+--------------------+-------+-----+--------+-------------------------------------+
|                 id|           user|           fullname| url|           timestamp|replies|likes|retweets|                                 text|
+-------------------+---------------+-------------------+----+--------------------+-------+-----+--------+-------------------------------------+
|1132977055300300800|   KamdemAbdiel|      Abdiel kamdem|null|2019-05-27 11:49:...|      0|    0|       0|                 È appena uscito u...|
|1132977073402736640|      bitcointe|          Bitcointe|null|2019-05-27 11:49:...|      0|    0|       0|                 Cardano: Digitize...|
|1132977023893139456|      3eyedbran|Bran - 3 Eyed Raven|null|2019-05-27 11:49:...|      0|    2|       1|                 Another Test twee...|
|1132977089089556481|  DetroitCrypto|        J. Scardina|null|2019-05-27 11:49:...|      0|    0|       0|                 Current

In [30]:
df = df.withColumn("id", df["id"].cast(IntegerType()))
df = df.withColumn("replies", df["replies"].cast(IntegerType()))
df = df.withColumn("likes", df["likes"].cast(IntegerType()))
df = df.withColumn("retweets", df["retweets"].cast(IntegerType()))
df = df.withColumn("timestamp",to_timestamp("timestamp"))

In [31]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- user: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- url: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- replies: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- retweets: integer (nullable = true)
 |-- text: string (nullable = true)



In [34]:
file = '../input/bitcoin-tweets-16m-tweets-with-sentiment-tagged/mbsa.csv'
df2 = spark.read.csv(file, sep=",", header=True,multiLine=True)

In [35]:
df2.show()

+----------+-------------------------------------+---------+
|      Date|                                 text|Sentiment|
+----------+-------------------------------------+---------+
|2019-05-27|                 È appena uscito u...| Positive|
|2019-05-27|                 Cardano: Digitize...| Positive|
|2019-05-27|                 Another Test twee...| Positive|
|2019-05-27|                 Current Crypto Pr...| Positive|
|2019-05-27|                 Spiv (Nosar Baz):...| Positive|
|2019-05-27|                 #btc inceldiği ye...| Positive|
|2019-05-27|                 @nwoodfine We hav...| Positive|
|2019-05-27|                 @pedronauck como ...| Positive|
|2019-05-27|ブラジルはまぁ置いといてもドイツは...| Positive|
|2019-05-27|                 CHANGE IS COMING....| Positive|
|2019-05-27|                 One of the useful...| Positive|
|2019-05-27|                 Invested my Life ...| Positive|
|2019-05-27|                 Bitcoin Price Hit...| Positive|
|2019-05-27|  【毎日プレゼント企画】\n１万円分...| Posit

In [36]:
result_df = df.join(df2, ("text")).drop('Date')

In [37]:
result_df.show()

+--------------------+----+---------------+--------------------+--------------------+-------------------+-------+-----+--------+----------+---------+
|                text|  id|           user|            fullname|                 url|          timestamp|replies|likes|retweets|      Date|Sentiment|
+--------------------+----+---------------+--------------------+--------------------+-------------------+-------+-----+--------+----------+---------+
|\n#PayWithLitecoi...|null|LTCTheRightCoin|               Randy|/LTCTheRightCoin/...|2019-01-17 14:28:52|      0|   10|       3|2019-01-17| Positive|
|\n#TRIG #1stblood...|null|       tinwacao|           Tinwafeng|/tinwacao/status/...|2017-12-09 05:06:53|      0|    0|       0|2017-12-09| Negative|
|          \n#bitcoin|null|  TheCryptoShiv|     The Crypto Shiv|/TheCryptoShiv/st...|2018-06-02 20:37:02|      0|    0|       0|2018-06-02| Positive|
|\n$BTC #Bitcoin p...|null|       smokin35|              G.love|/smokin35/status/...|2018-02-06 18:5

In [38]:
result_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- user: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- url: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- replies: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- retweets: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [55]:
file = '../input/392-crypto-currency-pairs-at-minute-resolution/btcusd.csv'
btc = spark.read.csv(file, sep=",", header=True,multiLine=True)
from pyspark.sql import functions as F
btc = btc.withColumn("timestamp", F.to_utc_timestamp(F.from_unixtime(F.col("time")/1000,'yyyy-MM-dd HH:mm:ss'),'IST')).drop('time')

In [56]:
btc.show()

+--------+--------+--------+--------+-----------+-------------------+
|    open|   close|    high|     low|     volume|          timestamp|
+--------+--------+--------+--------+-----------+-------------------+
|   93.25|    93.3|    93.3|   93.25|       93.3|2013-03-31 18:37:00|
|   100.0|   100.0|   100.0|   100.0|       93.3|2013-03-31 18:38:00|
|    93.3|    93.3|    93.3|    93.3|33.67686227|2013-03-31 18:39:00|
|   93.35|   93.47|   93.47|   93.35|       20.0|2013-03-31 18:41:00|
|   93.47|   93.47|   93.47|   93.47| 2.02162704|2013-03-31 18:42:00|
|93.03001|   93.03|93.03001|   93.03|     4.9854|2013-03-31 18:43:00|
|93.04399|93.04399|93.04399|93.04399|12.68950068|2013-03-31 18:56:00|
|93.34999|   93.35|   93.35|93.34999|        5.0|2013-03-31 18:58:00|
|93.19999|   93.35|   93.35|93.19999|       40.0|2013-03-31 19:05:00|
|   93.25|    93.3|    93.3|   93.25|       18.3|2013-03-31 19:23:00|
|    93.1|    93.1|    93.1|    93.1|22.31049932|2013-03-31 19:24:00|
|93.37999|93.37999|9

In [51]:
btc.printSchema()

root
 |-- time: timestamp (nullable = true)
 |-- open: string (nullable = true)
 |-- close: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- volume: string (nullable = true)



In [57]:
data = result_df.join(btc, ("timestamp"))

In [58]:
data.show()

+-------------------+------------------------------------+----+---------------+--------------------+--------------------+-------+-----+--------+----------+---------+------+------+------+------+------------------+
|          timestamp|                                text|  id|           user|            fullname|                 url|replies|likes|retweets|      Date|Sentiment|  open| close|  high|   low|            volume|
+-------------------+------------------------------------+----+---------------+--------------------+--------------------+-------+-----+--------+----------+---------+------+------+------+------+------------------+
|2014-10-09 19:05:00|現在の1ビットコインあたりの値段は...|null|current_price_v|        いまいくら？|/current_price_v/...|      0|    0|       0|2014-10-09| Positive|363.86|363.86|363.87|363.86|            2.4613|
|2014-10-24 02:30:00|現在の1ビットコインあたりの値段は...|null|current_price_v|        いまいくら？|/current_price_v/...|      0|    0|       0|2014-10-24| Negative|359.71|359.99|359.99| 359.7

In [ ]:
data.count()